<a href="https://colab.research.google.com/github/hoanglambinh/ISODS-Entrance/blob/main/ISODS_Test_2%2B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os
import glob
import torch
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModel
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

# === CONFIGURATION ===
BASE_PATH = "/content/drive/MyDrive/BoPhapDienDienTu/vbpl"
MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"

# === STEP 1: Load Vietnamese Bi-Encoder Model ===
print("Loading embedding model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def embed_text(text):
    """Generate embeddings for given text chunk."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=2000)
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

# === STEP 2: Load and Parse HTML Files ===
def load_html_files(base_path):
    """Load all full_ItemID.html files and extract text."""
    html_files = glob.glob(os.path.join(base_path, "full_*.html"))
    documents = []

    for file_path in html_files:
        with open(file_path, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")
            text = soup.get_text(separator=" ").strip()

        if text:  # Ensure non-empty text
            documents.append(Document(page_content=text, metadata={"source": file_path}))

    print(f"Found {len(documents)} documents.")
    return documents

# === STEP 3: Chunk Text ===
def chunk_text(text, tokenizer, max_tokens=2000, overlap=20):
    """Chunk text into segments with overlap."""
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []

    for i in range(0, len(tokens), max_tokens - overlap):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk))

    return chunks

# === STEP 4: Process Documents ===
def process_documents(documents, tokenizer):
    """Chunk and prepare documents for embedding."""
    chunked_docs = []

    for doc in documents:
        chunks = chunk_text(doc.page_content, tokenizer)
        for i, chunk in enumerate(chunks):
            chunked_docs.append(Document(page_content=chunk, metadata={"source": doc.metadata["source"], "chunk_id": i}))

    print(f"Generated {len(chunked_docs)} text chunks.")
    return chunked_docs

# === STEP 5: Create and Populate ChromaDB ===
def create_vector_db(documents):
    """Embed documents and store them in ChromaDB."""
    embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)
    db = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

    texts = [doc.page_content for doc in documents]
    metadatas = [doc.metadata for doc in documents]

    db.add_texts(texts=texts, metadatas=metadatas)
    db.persist()

    print("ChromaDB successfully created and populated!")

# === RUN THE PIPELINE ===
documents = load_html_files(BASE_PATH)
chunked_docs = process_documents(documents, tokenizer)
create_vector_db(chunked_docs)

print("Vector database is ready.")


Loading embedding model...
Found 51 documents.
Generated 157 text chunks.
ChromaDB successfully created and populated!
Vector database is ready.


In [24]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# === CONFIGURATION ===
CHROMA_DB_PATH = "chroma_db"  # Path to stored vector DB
MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"  #Embedding model
TOP_K = 5  # Number of results to return

# === LOAD EMBEDDINGS & VECTOR DB ===
print(" Loading embeddings and ChromaDB...")
embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)
db = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)

# === SEMANTIC SEARCH FUNCTION ===
def semantic_search(query, top_k=TOP_K):
    """Search for the most relevant documents using semantic similarity."""
    print(f"\n Searching for: {query}\n")
    results = db.similarity_search(query, k=top_k)

    for i, doc in enumerate(results):
        print(f" Rank {i+1}")
        print(f" Content: {doc.page_content[:500]}...")  # Show first 500 characters
        print(f" Source: {doc.metadata['source']}\n")  # File path in metadata
        print("=" * 50)

# === USER INPUT ===
while True:
    user_query = input("\nEnter search query (or 'exit' to quit): ")
    if user_query.lower() == "exit":
        print("End")
        break
    semantic_search(user_query)


 Loading embeddings and ChromaDB...

Enter search query (or 'exit' to quit): 5

 Searching for: 5

 Rank 1
 Content: thời gian chuyển từ cơ sở quản lý này đến cơ sở quản lý khác hoặc từ cơ sở quản lý ra ngoài cộng đồng. b) Trường hợp cơ sở quản lý chưa đủ điều kiện thực hiện việc điều trị bằng thuốc kháng HIV theo quy định tại Thông tư số 09/2011/TT-BYT, việc dự trù, phân phối, quản lý và sử dụng thuốc kháng HIV miễn phí thực hiện như sau: - Hàng năm, cơ sở điều trị được chỉ định lập dự trù thuốc kháng HIV cho các đối tượng được quản lý tại cơ sở quản lý và gửi về Trung tâm Phòng, chống HIV/AIDS cấp tỉnh. Trun...
 Source: /content/drive/MyDrive/BoPhapDienDienTu/vbpl/full_66817.html

 Rank 2
 Content: thời gian chuyển từ cơ sở quản lý này đến cơ sở quản lý khác hoặc từ cơ sở quản lý ra ngoài cộng đồng. b) Trường hợp cơ sở quản lý chưa đủ điều kiện thực hiện việc điều trị bằng thuốc kháng HIV theo quy định tại Thông tư số 09/2011/TT-BYT, việc dự trù, phân phối, quản lý và sử dụng thuốc k